# **Data Preprocessing**

In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
User_id = []
Bus_id = []
Star = []
Useful = []
Funny = []
Cool = []
Text = []
I=1

In [3]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ndavidson/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ndavidson/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ndavidson/nltk_data...


In [4]:
DF_Final = pd.read_csv('./Downloads/Data_Final')

In [5]:
documents = []
X = DF_Final.Review
stemmer = WordNetLemmatizer()
for i in range(0, len(X)):
    # Remove all the special characters, like parathesis
    document = re.sub(r'\W', ' ', str(X[i]))
    # remove all single characters: like a, b, c, d
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=0.1, max_df=0.7, 
stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [66]:
f_n = open("./Downloads/negative-words.txt", "r")
Negative_words = f_n.readlines()
f_p = open("./Downloads/positive-words.txt", "r")
Positive_words = f_p.readlines()
Voca = [i.rstrip('\n') for i in Positive_words] + [i.rstrip('\n') for i in 
Negative_words]
vectorizer = CountVectorizer(vocabulary=np.unique(Voca))
X = vectorizer.fit_transform(documents).toarray()

In [78]:
DF_Final.head()

,Unnamed: 0,User_id,Bus_id,Star,Useful,Cool,Funny,Review,State,City,Bus_Ave_Star,User_Review_count,User_Useful_count,User_Funny_count,User_Cool_count,Elite,User_Fans,Users_Ave_Star
0,0,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",CA,Goleta,4.0,922,1687,694,1070,"2015,2016,2017,2018,2019,20,20,2021",51,4.20
1,1,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,0,0,0,We were a bit weary about trying the Shellfish...,CA,Santa Barbara,4.0,338,800,144,353,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",30,4.12
2,2,Uk3X2AypU8AqvcYEVf7s6Q,eL4lyE7LNoXEMvpcJ8WNVw,3.0,5,0,0,This was a tough one! On the merits of the w...,CA,Santa Barbara,4.0,431,2126,1245,1476,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",76,4.05
3,3,LcqNuhqaYt5ekKzaRirmIg,SZU9c8V2GuREDN5KgyHFJw,5.0,2,1,0,"I love trying fresh seafood on piers, wharfs a...",CA,Santa Barbara,4.0,258,452,125,183,"2015,2016,2017",37,3.99
4,4,gasLVm0KRwrVhPGRcqATjw,CHh0ZFrQcsk4boOItr2Zuw,4.0,1,2,0,I stopped in because I was hungry for some sna...,CA,Carpinteria,3.0,1638,2658,933,2297,"2014,2015,2016,2017,2018,2019,20,20,2021",105,3.79


# **Train a Naive Bayes Classifier**

In [87]:
#80-10-10 split of train test
test = DF_Final[(DF_Final['Star'] == 1.0) | (DF_Final['Star'] == 5.0)]
Y = DF_Final['Star']
from sklearn.model_selection import train_test_split
X_train, X_rem, Y_train, Y_rem = train_test_split(X, Y, train_size=0.8)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem, Y_rem, test_size=0.5)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_valid.shape)
print(Y_test.shape)
print(test.shape)


(43076, 6786)
(5384, 6786)
(5385, 6786)
(43076,)
(5384,)
(5385,)
(23285, 18)


In [88]:
test_size = np.linspace(0.1, 1, num=9, endpoint=False)
random_state = np.arange(0, 43)
grid_results= []
for testsize in test_size:
    for randomstate in random_state:
        try:
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=testsize, random_state=randomstate)
            mnb = MultinomialNB()
            mnb.fit(X_train, y_train)
            y_test_pred = mnb.predict(X_test)     
            grid_results.append([testsize, randomstate, mean_squared_error(y_test, y_test_pred)])
            grid_frame = pd.DataFrame(grid_results)
            grid_frame.rename(columns={0:'Test Size', 1:'Random State', 2:'MSE of Test'}, inplace=True)
        except Exception:
            print(Exception.with_traceback())
            print('error')
            continue
min_test_mse = grid_frame[grid_frame['MSE of Test'] == grid_frame['MSE of Test'].min()]
min_test_mse

TypeError: unbound method BaseException.with_traceback() needs an argument

In [82]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, Y_train)

MultinomialNB()

In [85]:
preds = classifier.predict(X_rem)

In [86]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(Y_rem, preds))
print('\n')
print(classification_report(Y_rem, preds))

[[ 131  108   72   48   21]
 [  41  128  279  141   81]
 [  25   86  548  759  252]
 [  14   43  304 1947 1444]
 [  28   24   77 1059 3109]]


              precision    recall  f1-score   support

         1.0       0.55      0.34      0.42       380
         2.0       0.33      0.19      0.24       670
         3.0       0.43      0.33      0.37      1670
         4.0       0.49      0.52      0.51      3752
         5.0       0.63      0.72      0.68      4297

    accuracy                           0.54     10769
   macro avg       0.49      0.42      0.44     10769
weighted avg       0.53      0.54      0.53     10769

